In [1]:
from yugiquery import *
header('Bandai')

<div align='center'>
    <br>
    <!-- Pre Style needed to fix HTML formatting -->
    <pre style="line-height: var(--jp-code-line-height); font-family: var(--jp-code-font-family)">
    ██    ██ ██    ██  ██████  ██  ██████  ██    ██ ███████ ██████  ██    ██ 
     ██  ██  ██    ██ ██       ██ ██    ██ ██    ██ ██      ██   ██  ██  ██  
      ████   ██    ██ ██   ███ ██ ██    ██ ██    ██ █████   ██████    ████   
       ██    ██    ██ ██    ██ ██ ██ ▄▄ ██ ██    ██ ██      ██   ██    ██    
       ██     ██████   ██████  ██  ██████   ██████  ███████ ██   ██    ██    
                                      ▀▀                                     
    </pre>
</div>
<div align='right'>
    Bandai - Execution started 20/02/2023 18:21 UTC <br>
    By <b>Guilherme Ruiz</b>
</div>

---

# Data aquisition

## Fetch online data

In [2]:
timestamp = pd.Timestamp.now()

In [3]:
bandai_df = fetch_bandai()

# Check changes

## Load previous data

In [4]:
# Get list of files
files_list = sorted(glob.glob('../data/all_bandai_*.csv'), key=os.path.getctime, reverse=True)
# Get second newest file if exist
if files_list:
    # Load csv avoiding converting "NA" to NaN
    previous_df = pd.read_csv(files_list[0], dtype=object, keep_default_na=False, na_values=[''])
    # Force dtypes to match current df
    previous_df = previous_df.astype(bandai_df[previous_df.columns.intersection(bandai_df.columns)].dtypes.to_dict())
    previous_ts = pd.to_datetime(os.path.basename(files_list[0]).split('_')[-1].rstrip('.csv'))
    print('File loaded')
else:
    previous_df = None
    print('No older files')

File loaded


## Generate changelog

In [5]:
if previous_df is None:
    changelog=None
    print('Skipped')
else:
    changelog = generate_changelog(previous_df, bandai_df, col = 'Name')
    if not changelog.empty:
        display(changelog)
        changelog.to_csv(f'../data/bandai_changelog_{timestamp.isoformat(timespec="minutes")}_{previous_ts.isoformat(timespec="minutes")}.csv', index = True)
        print('Changelog saved')

No changes


## Save data

In [6]:
if changelog is not None and changelog.empty:
    print('No changes. New data not saved')
else:
    bandai_df.to_csv(f'../data/all_bandai_{timestamp.isoformat(timespec="minutes")}.csv', index=False)
    print('Data saved')

No changes. New data not saved


# Data visualization

In [7]:
bandai_df

,Name,Card type,Monster type,DEF,Set,Card number,Ability,Rule,ATK,Level,Page name,Page URL
0,Aphrodite,Monster Card,White Magic,2300,Yu-Gi-Oh! Bandai OCG: 1st Generation,22,NaN,4,1000,5,Aphrodite,https://yugipedia.com/wiki/Aphrodite
1,Armored Basic Insect with Laser Cannon,Monster Card,Insect,1300,Yu-Gi-Oh! Bandai OCG: 1st Generation,17,NaN,8,2400,6,Armored Basic Insect with Laser Cannon,https://yugipedia.com/wiki/Armored_Basic_Insec...
2,Armored Lizard,Monster Card,Beast-Warrior,2000,Yu-Gi-Oh! Bandai OCG: 2nd Generation,73,NaN,6,1800,5,Armored Lizard (Bandai),https://yugipedia.com/wiki/Armored_Lizard_(Ban...
3,Armored Zombie,Monster Card,Fiend,0,Yu-Gi-Oh! Bandai OCG: 2nd Generation,80,NaN,11,2400,4,Armored Zombie (Bandai),https://yugipedia.com/wiki/Armored_Zombie_(Ban...
4,Axe Raider,Monster Card,Warrior,1500,Yu-Gi-Oh! Bandai OCG: 2nd Generation,60,NaN,7,2300,6,Axe Raider (Bandai),https://yugipedia.com/wiki/Axe_Raider_(Bandai)
...,...,...,...,...,...,...,...,...,...,...,...,...
115,The Wicked Worm Beast,Monster Card,Fiend,600,Yu-Gi-Oh! Bandai OCG: 3rd Generation,94,NaN,5,2400,5,The Wicked Worm Beast (Bandai),https://yugipedia.com/wiki/The_Wicked_Worm_Bea...
116,"Winged Dragon, Guardian of the Fortress #1",Monster Card,Dragon,1600,Yu-Gi-Oh! Bandai OCG: 1st Generation,23,NaN,5,1700,4,"Winged Dragon, Guardian of the Fortress 1 (Ban...","https://yugipedia.com/wiki/Winged_Dragon,_Guar..."
117,Yellow Hecate,Monster Card,Black Magic,1500,Yu-Gi-Oh! Bandai OCG: 3rd Generation,98,NaN,NaN,1500,4,Yellow Hecate,https://yugipedia.com/wiki/Yellow_Hecate
118,Yugi Muto,Character Card,NaN,NaN,Yu-Gi-Oh! Bandai OCG: 1st Generation,1,Look Ahead,1,NaN,6,Yugi Muto (Bandai),https://yugipedia.com/wiki/Yugi_Muto_(Bandai)


# Epilogue

In [8]:
benchmark('bandai',timestamp)

In [9]:
footer()

<div align='right'>
    Execution completed 20/02/2023 18:21 UTC
</div>

## HTML export

In [10]:
# Save notebook on disck before generating HTML report
save_notebook()

Notebook saved to disk


In [11]:
! jupyter nbconvert Bandai.ipynb --output-dir='../' --to=HTML --TagRemovePreprocessor.enabled=True --TagRemovePreprocessor.remove_cell_tags='exclude' --TemplateExporter.exclude_input=True --TemplateExporter.exclude_input_prompt=True --TemplateExporter.exclude_output_prompt=True

[NbConvertApp] Converting notebook Bandai.ipynb to HTML


[NbConvertApp] Writing 590112 bytes to ../Bandai.html


## Git

In [ ]:
! git add "../*[Bb]andai*"

In [ ]:
! git commit -m {"'Bandai update-" + timestamp.isoformat() + "'"}